In [1]:
import sys
sys.path.append('../')

In [2]:
import warnings
import logging
import time
import psutil
import os

import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime

from its_logging.logger_config import logger
from utils.its_utils import clip_to_california, get_wfr_tf_template
from utils.gdf_utils import repair_geometries, show_columns, verify_gdf_columns
from utils.add_common_columns import add_common_columns
from utils.enrich_polygons import enrich_polygons
from utils.keep_fields import keep_fields
from utils.assign_domains import assign_domains
from utils.save_gdf_to_gdb import save_gdf_to_gdb

In [3]:
a_reference_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb"
start_year, end_year = 2020, 2025

process = psutil.Process(os.getpid())

In [4]:
from enrich.enrich_NPS import enrich_NPS_from_gdb

In [5]:
nps_gdb_path = r'D:\WORK\wildfire\Interagency-Tracking-System\2023\NPS_2023\New_NPS_2023_20240625_ReisThomasViaUpload_1.gdb'
nps_layer_name = 'NPS_2023_20240625_ReisThomasViaUpload2'
output_gdb_path =  r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\NPS_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"NPS_enriched_{datetime.today().strftime('%Y%m%d')}"



In [6]:
NPS_gpd = enrich_NPS_from_gdb(nps_gdb_path,
                    nps_layer_name,
                    a_reference_gdb_path,
                    start_year,
                    end_year,
                    output_gdb_path,
                    output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
logger.info(f"Memory usage: {memory_usage:.2f} MB")

2025-01-27 10:04:16,310 INFO  [enrich.enrich_NPS     ]  Load the NPS data into a GeoDataFrame
2025-01-27 10:04:16,913 INFO  [enrich.enrich_NPS     ]     time for loading NPS_2023_20240625_ReisThomasViaUpload2: 0.6021413803100586
2025-01-27 10:04:16,914 INFO  [enrich.enrich_NPS     ]     all required columns are present.
2025-01-27 10:04:17,393 INFO  [enrich.enrich_NPS     ]  Performing Standardization...
2025-01-27 10:04:17,393 INFO  [enrich.enrich_NPS     ]     step 1/11 select after 1995
2025-01-27 10:04:17,406 INFO  [enrich.enrich_NPS     ]     step 2/11 repairing geometry
2025-01-27 10:04:21,077 INFO  [enrich.enrich_NPS     ]     step 3/11 clip features by CA
2025-01-27 10:04:32,056 INFO  [enrich.enrich_NPS     ]     step 4/11 dissolve to implement multipart polygons
2025-01-27 10:04:32,786 INFO  [enrich.enrich_NPS     ]     step 5/11 rename and add fields
2025-01-27 10:04:32,813 INFO  [enrich.enrich_NPS     ]     step 6/11 import attributes
2025-01-27 10:04:32,839 INFO  [enrich.en

2025-01-27 10:15:27,915 INFO  [utils.counts_to_mas   ]              counts step 8/8: set to 'NO' if Org is 'USFS' and Status is 'Active'
2025-01-27 10:15:27,919 INFO  [utils.enrich_polygons ]           Crosswalk Complete. Continuing Enrichment...
2025-01-27 10:15:27,919 INFO  [utils.enrich_polygons ]              enrich step 29/32 Calculating Latitude and Longitude...
2025-01-27 10:15:27,939 INFO  [utils.enrich_polygons ]              enrich step 30/32 calculate treatment acres
2025-01-27 10:15:27,942 INFO  [utils.enrich_polygons ]              enrich step 31/32 removing unnecessary fields
2025-01-27 10:15:27,944 INFO  [utils.enrich_polygons ]              enrich step 32/32 delete if County is Null
2025-01-27 10:15:28,210 INFO  [enrich.enrich_NPS     ]     step 9/11 adding treatment ID
2025-01-27 10:15:28,218 INFO  [enrich.enrich_NPS     ]     step 10/11 Assign Domains...
2025-01-27 10:15:28,499 INFO  [utils.assign_domains  ]        Created domain 'D_OBJECTIVE' with 27 values
2025-01-2

In [20]:
NPS_reference = gpd.read_file(r'D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb', driver='openFileGDB', layer='nps_flat_fuels_enriched2023_20240925')

In [21]:
NPS_reference.columns

Index(['PROJECTID_USER', 'AGENCY', 'ORG_ADMIN_p', 'PROJECT_CONTACT',
       'PROJECT_EMAIL', 'ADMINISTERING_ORG', 'PROJECT_NAME', 'PROJECT_STATUS',
       'PROJECT_START', 'PROJECT_END', 'PRIMARY_FUNDING_SOURCE',
       'PRIMARY_FUNDING_ORG', 'IMPLEMENTING_ORG', 'LATITUDE', 'LONGITUDE',
       'BatchID_p', 'Val_Status_p', 'Val_MSG_p', 'Val_RunDate_p',
       'Review_Status_p', 'Review_MSG_p', 'Review_RunDate_p',
       'Dataload_Status_p', 'Dataload_Msg_p', 'TRMTID_USER', 'PROJECTID',
       'PROJECTNAME_', 'ORG_ADMIN_t', 'PRIMARY_OWNERSHIP_GROUP',
       'PRIMARY_OBJECTIVE', 'SECONDARY_OBJECTIVE', 'TERTIARY_OBJECTIVE',
       'TREATMENT_STATUS', 'COUNTY', 'IN_WUI', 'REGION', 'TREATMENT_AREA',
       'TREATMENT_START', 'TREATMENT_END', 'RETREATMENT_DATE_EST',
       'TREATMENT_NAME', 'BatchID', 'Val_Status_t', 'Val_MSG_t',
       'Val_RunDate_t', 'Review_Status_t', 'Review_Msg_t', 'Review_RunDate_t',
       'Dataload_Status_t', 'Dataload_Msg_t', 'ACTIVID_USER', 'TREATMENTID_',
       '

In [22]:
NPS_reference[['ACTIVITY_QUANTITY', 'ADMINISTERING_ORG', 'COUNTS_TO_MAS', 'PRIMARY_OBJECTIVE', "Year_txt"]].groupby(['ADMINISTERING_ORG', 'COUNTS_TO_MAS', 'PRIMARY_OBJECTIVE', "Year_txt"]).sum()

ACTIVITY_QUANTITY
ADMINISTERING_ORG COUNTS_TO_MAS PRIMARY_OBJECTIVE     Year_txt                      
NPS               NO            ECO_RESTOR            2023       481.000000000000000
                                NOT_DEFINED           2021      1018.000000000000000
                  YES           ECO_RESTOR            2020      1115.000000000000000
                                                      2021       451.000000000000000
                                                      2023       246.000000000000000
                                OTHER_FUELS_REDUCTION 2020      1050.000000000000000
                                                      2021      3743.000000000000000
                                                      2022      3609.000000000000000
                                                      2023      4508.000000000000000
                                PRESCRB_FIRE          2020     12051.000000000000000
                                                      2021      3480.000000000000000

In [34]:
NPS_reference.geometry

0      MULTIPOLYGON Z (((-118551.063 270862.26 0, -11...
1      MULTIPOLYGON Z (((-130560.279 416575.149 0, -1...
2      MULTIPOLYGON Z (((234219.762 -110382.406 0, 23...
3      MULTIPOLYGON Z (((-213822.819 291264.476 0, -2...
4      MULTIPOLYGON Z (((-222842.298 297066.356 0, -2...
                             ...                        
155    MULTIPOLYGON Z (((262405.168 -192693.665 0, 26...
156    MULTIPOLYGON Z (((33279.901 -32154.605 0, 3327...
157    MULTIPOLYGON Z (((-128341.67 283518.877 0, -12...
158    MULTIPOLYGON Z (((-335795.604 422976.427 0, -3...
159    MULTIPOLYGON Z (((-136538.184 260521.098 0, -1...
Name: geometry, Length: 160, dtype: geometry

In [37]:
from shapely import MultiPolygon, Polygon
NPS_gpd.geometry = NPS_gpd.geometry.apply(lambda x: MultiPolygon([x]) if isinstance(x, Polygon) else x)

In [44]:
NPS_gpd.geometry.apply(lambda x: type(x))

AttributeError: 'property' object has no attribute 'get'

In [39]:
NPS_gpd.to_file(output_gdb_path, 
                driver="OpenFileGDB", 
                layer=output_layer_name)

In [8]:
from enrich.enrich_BLM import enrich_BLM

In [9]:
blm_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\2023\BLM_2023\BLM_2010_2023_fromReisThomasViaUpload.gdb"
blm_input_layer_name = "BLM_2010_2023_fromReisThomasViaUpload"
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\BLM_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"BLM_enriched_{datetime.today().strftime('%Y%m%d')}"

In [10]:

cnra_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\2023\CNRA_2023\CNRA_Tracker_Data_Export_20241120.gdb"
cnra_polygon_layer_name = "TREATMENT_POLY"
cnra_line_layer_name = "TREATMENT_LINE"
cnra_point_layer_name = "TREATMENT_POINT"
cnra_project_polygon_layer_name = "PROJECT_POLY"
cnra_activity_layer_name = "ACTIVITIES"
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"CNRA_enriched_{datetime.today().strftime('%Y%m%d')}"


In [11]:
# temp file path
caltrans_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\2023\CALTRANS_2023\Caltrans_Vegetation_Management_20_23.gdb"
tree_activity_layer_name = "Caltrans_Vegetation_Management_Trees_ActivitiesTable_20_23"
tree_treatment_layer_name = "Caltrans_Vegetation_Management_Trees_Treatments_20_23"
road_activity_layer_name = "Caltrans_Vegetation_Management_RoadsideLandscape_ActivitiesTable_20_23"
road_treatment_layer_name = "Caltrans_Vegetation_Management_RoadsideLandscape_Treatments_20_23"
start_year = 2021
end_year = 2023
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CalTRANS_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"CalTRANS_enriched_{datetime.today().strftime('%Y%m%d')}"